# Testing for E1 - using all spindles, training with all data
fistly define the UUID of the experiment to use

calculating with classical F1 to compare with our F1* in the reviewers' response

In [3]:
experimentId="5388ca14-a315-4598-97c2-d44175b24937"

-------------------------------------------------------

In [4]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## Overrride metrics to do classical F1 instead of F1*

In [5]:
def annotationPairToMetrics(annotations,detections,thresIoU=0.3):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1 the old way
    N=len(annotations)
    M=len(detections)
    TP=np.sum(np.max(binarized,axis=0))
    FN=N-TP
    FP=M-TP

    outF1=(2*TP)/(2*TP+FN+FP)
    recall=TP/N
    precision=TP/M

    #calculateF1*
    """ outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections) """
    return outF1,recall,precision

## load mass

In [6]:
samplerate=200  #Should rethink this

In [7]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [8]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [9]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [10]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [11]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [12]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [13]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,d51d037b-304a-48df-970e-58aac592ac87,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,50ebd176-f601-4f02-bcd5-e0c07a362a35,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,ad0e1cc1-5a53-45b1-b13e-ca760e58c787,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,953522b6-a687-4555-b7bd-64e50323f539,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,fdff7a52-c874-434c-b79a-4652e6521b61,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,a99170bc-4dc5-4b27-84df-f565c6f142c5,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,d4f86014-a983-4a56-940a-a53396539084,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,b9ba925c-8862-41ae-85ac-87efe667dc99,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,9a796814-2401-4dbb-811c-695c7c0888bb,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,3be685c1-02ea-4a2f-8478-ca265ccc3196,0.016460


In [14]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.058320  0.004520  0.052347  0.054965   
union                    15.0  0.059557  0.004579  0.053738  0.055972   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.056746  0.061623  0.067005  
union          0.058222  0.062711  0.068526

In [15]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [16]:
hyperClose=0.1
hyperDuration=minDuration
hyperThres=0.3
hyperDepth=30

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [17]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,d51d037b-304a-48df-970e-58aac592ac87,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,50ebd176-f601-4f02-bcd5-e0c07a362a35,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,ad0e1cc1-5a53-45b1-b13e-ca760e58c787,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,953522b6-a687-4555-b7bd-64e50323f539,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,fdff7a52-c874-434c-b79a-4652e6521b61,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,a99170bc-4dc5-4b27-84df-f565c6f142c5,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,d4f86014-a983-4a56-940a-a53396539084,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,b9ba925c-8862-41ae-85ac-87efe667dc99,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,9a796814-2401-4dbb-811c-695c7c0888bb,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,3be685c1-02ea-4a2f-8478-ca265ccc3196,0.016460


In [18]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [19]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [20]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [21]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [23]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [24]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.678188 NaN  0.678188  0.678188  0.678188            1.0   
0002     1.0  0.789149 NaN  0.789149  0.789149  0.789149            1.0   
0003     1.0  0.658462 NaN  0.658462  0.658462  0.658462            1.0   
0005     1.0  0.668192 NaN  0.668192  0.668192  0.668192            1.0   
0006     1.0  0.673401 NaN  0.673401  0.673401  0.673401            1.0   
0007     1.0  0.640769 NaN  0.640769  0.640769  0.640769            1.0   
0009     1.0  0.769947 NaN  0.769947  0.769947  0.769947            1.0   
0010     1.0  0.768258 NaN  0.768258  0.768258  0.768258            1.0   
0011     1.0  0.686820 NaN  0.686820  0.686820  0.686820            1.0   
0012     1.0  0.725726 NaN  0.725726  0.725726  0.725726            1.0   
0013     1.0  0.677320 NaN  0.677320  0.677320  0.677320            1.0   
0014     1.0  0.599034 NaN  0.599034  0.599034  0.599034            1.0   
0017     1.0  0.737218 NaN  0.737218  0.737218  0.737218            1.0   
0018     1.0  0.740417 NaN  0.740417  0.740417  0.740417            1.0   
0019     1.0  0.571691 NaN  0.571691  0.571691  0.571691            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.897476 NaN  0.897476  0.897476  0.897476         1.0  0.545019 NaN   
0002  0.744186 NaN  0.744186  0.744186  0.744186         1.0  0.839895 NaN   
0003  0.587912 NaN  0.587912  0.587912  0.587912         1.0  0.748252 NaN   
0005  0.547842 NaN  0.547842  0.547842  0.547842         1.0  0.856305 NaN   
0006  0.680272 NaN  0.680272  0.680272  0.680272         1.0  0.666667 NaN   
0007  0.517544 NaN  0.517544  0.517544  0.517544         1.0  0.841009 NaN   
0009  0.839130 NaN  0.839130  0.839130  0.839130         1.0  0.711302 NaN   
0010  0.707627 NaN  0.707627  0.707627  0.707627         1.0  0.840252 NaN   
0011  0.541985 NaN  0.541985  0.541985  0.541985         1.0  0.937294 NaN   
0012  0.729345 NaN  0.729345  0.729345  0.729345         1.0  0.722144 NaN   
0013  0.528986 NaN  0.528986  0.528986  0.528986         1.0  0.941261 NaN   
0014  0.435783 NaN  0.435783  0.435783  0.435783         1.0  0.957865 NaN   
0017  0.835580 NaN  0.835580  0.835580  0.835580         1.0  0.659574 NaN   
0018  0.850613 NaN  0.850613  0.850613  0.850613         1.0  0.655498 NaN   
0019  0.402329 NaN  0.402329  0.402329  0.402329         1.0  0.987302 NaN   

                                    
           min       50%       max  
test                                
0001  0.545019  0.545019  0.545019  
0002  0.839895  0.839895  0.839895  
0003  0.748252  0.748252  0.748252  
0005  0.856305  0.856305  0.856305  
0006  0.666667  0.666667  0.666667  
0007  0.841009  0.841009  0.841009  
0009  0.711302  0.711302  0.711302  
0010  0.840252  0.840252  0.840252  
0011  0.937294  0.937294  0.937294  
0012  0.722144  0.722144  0.722144  
0013  0.941261  0.941261  0.941261  
0014  0.957865  0.957865  0.957865  
0017  0.659574  0.659574  0.659574  
0018  0.655498  0.655498  0.655498  
0019  0.987302  0.987302  0.987302

In [26]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.678188,0.897476,0.545019
1,0002,0.789149,0.744186,0.839895
2,0003,0.658462,0.587912,0.748252
3,0005,0.668192,0.547842,0.856305
4,0006,0.673401,0.680272,0.666667
5,0007,0.640769,0.517544,0.841009
6,0009,0.769947,0.839130,0.711302
7,0010,0.768258,0.707627,0.840252
8,0011,0.686820,0.541985,0.937294
9,0012,0.725726,0.729345,0.722144


In [27]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           6.923060e-01
eventPrecision    6.564406e-01
eventRecall       7.939759e-01
dtype: float64

In [28]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_5776\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.062716
eventPrecision    0.159432
eventRecall       0.132645
dtype: float64

In [29]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [30]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [31]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,d51d037b-304a-48df-970e-58aac592ac87,0.018639,0.627405,0.841833,0.500038,0.625346,0.845816,0.496047,0.678188,0.897476,0.545019
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,50ebd176-f601-4f02-bcd5-e0c07a362a35,0.020055,0.733682,0.714589,0.753823,0.735542,0.719371,0.752455,0.789149,0.744186,0.839895
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,ad0e1cc1-5a53-45b1-b13e-ca760e58c787,0.021075,0.617334,0.551438,0.701114,0.633646,0.576858,0.702837,0.658462,0.587912,0.748252
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,953522b6-a687-4555-b7bd-64e50323f539,0.018331,0.598996,0.488493,0.774106,0.613069,0.506517,0.776391,0.668192,0.547842,0.856305
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,fdff7a52-c874-434c-b79a-4652e6521b61,0.019222,0.607848,0.648662,0.571866,0.610757,0.669309,0.561625,0.673401,0.680272,0.666667


In [32]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [33]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [34]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [35]:
dumpPickle('experimentModelsTest_N2_E1_temp.pkl',experimentModels_N2)

In [36]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [37]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [38]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [39]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [40]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.690534 NaN  0.690534  0.690534  0.690534            1.0   
0002     1.0  0.799828 NaN  0.799828  0.799828  0.799828            1.0   
0003     1.0  0.699346 NaN  0.699346  0.699346  0.699346            1.0   
0005     1.0  0.692764 NaN  0.692764  0.692764  0.692764            1.0   
0006     1.0  0.701754 NaN  0.701754  0.701754  0.701754            1.0   
0007     1.0  0.730476 NaN  0.730476  0.730476  0.730476            1.0   
0009     1.0  0.779798 NaN  0.779798  0.779798  0.779798            1.0   
0010     1.0  0.829814 NaN  0.829814  0.829814  0.829814            1.0   
0011     1.0  0.710828 NaN  0.710828  0.710828  0.710828            1.0   
0012     1.0  0.783410 NaN  0.783410  0.783410  0.783410            1.0   
0013     1.0  0.720000 NaN  0.720000  0.720000  0.720000            1.0   
0014     1.0  0.616079 NaN  0.616079  0.616079  0.616079            1.0   
0017     1.0  0.755177 NaN  0.755177  0.755177  0.755177            1.0   
0018     1.0  0.750246 NaN  0.750246  0.750246  0.750246            1.0   
0019     1.0  0.570356 NaN  0.570356  0.570356  0.570356            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.942053 NaN  0.942053  0.942053  0.942053         1.0  0.545019 NaN   
0002  0.764996 NaN  0.764996  0.764996  0.764996         1.0  0.837984 NaN   
0003  0.656442 NaN  0.656442  0.656442  0.656442         1.0  0.748252 NaN   
0005  0.581673 NaN  0.581673  0.581673  0.581673         1.0  0.856305 NaN   
0006  0.740741 NaN  0.740741  0.740741  0.740741         1.0  0.666667 NaN   
0007  0.645623 NaN  0.645623  0.645623  0.645623         1.0  0.841009 NaN   
0009  0.862891 NaN  0.862891  0.862891  0.862891         1.0  0.711302 NaN   
0010  0.819632 NaN  0.819632  0.819632  0.819632         1.0  0.840252 NaN   
0011  0.571721 NaN  0.571721  0.571721  0.571721         1.0  0.939394 NaN   
0012  0.857143 NaN  0.857143  0.857143  0.857143         1.0  0.721358 NaN   
0013  0.582964 NaN  0.582964  0.582964  0.582964         1.0  0.941261 NaN   
0014  0.454061 NaN  0.454061  0.454061  0.454061         1.0  0.957865 NaN   
0017  0.883191 NaN  0.883191  0.883191  0.883191         1.0  0.659574 NaN   
0018  0.877011 NaN  0.877011  0.877011  0.877011         1.0  0.655498 NaN   
0019  0.401055 NaN  0.401055  0.401055  0.401055         1.0  0.987013 NaN   

                                    
           min       50%       max  
test                                
0001  0.545019  0.545019  0.545019  
0002  0.837984  0.837984  0.837984  
0003  0.748252  0.748252  0.748252  
0005  0.856305  0.856305  0.856305  
0006  0.666667  0.666667  0.666667  
0007  0.841009  0.841009  0.841009  
0009  0.711302  0.711302  0.711302  
0010  0.840252  0.840252  0.840252  
0011  0.939394  0.939394  0.939394  
0012  0.721358  0.721358  0.721358  
0013  0.941261  0.941261  0.941261  
0014  0.957865  0.957865  0.957865  
0017  0.659574  0.659574  0.659574  
0018  0.655498  0.655498  0.655498  
0019  0.987013  0.987013  0.987013

In [41]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.690534,0.942053,0.545019
1,0002,0.799828,0.764996,0.837984
2,0003,0.699346,0.656442,0.748252
3,0005,0.692764,0.581673,0.856305
4,0006,0.701754,0.740741,0.666667
5,0007,0.730476,0.645623,0.841009
6,0009,0.779798,0.862891,0.711302
7,0010,0.829814,0.819632,0.840252
8,0011,0.710828,0.571721,0.939394
9,0012,0.783410,0.857143,0.721358


In [42]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.220274e-01
eventPrecision    7.094132e-01
eventRecall       7.939168e-01
dtype: float64

In [43]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_5776\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.067546
eventPrecision    0.167425
eventRecall       0.132763
dtype: float64

In [44]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [45]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [46]:
np.setdiff1d(experimentModels_N2.test,['0014','0019'])

array(['0001', '0002', '0003', '0005', '0006', '0007', '0009', '0010',
       '0011', '0012', '0013', '0017', '0018'], dtype=object)

In [47]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           7.461201e-01
eventPrecision    7.370023e-01
eventRecall       7.849046e-01
dtype: float64

In [48]:
annotationPairToGraph(gtAnnotations,processedAnnotations)